In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('./..')
from evaluator import eval_module
import os 
import numpy
import pickle
import numpy as np
import pandas as pd
import os
import sys
import tqdm
import pickle
import pathlib
from pathlib import Path
from evaluator import eval_module

In [2]:
import sys
sys.executable


'/home/ddatta/anaconda3/envs/graph1/bin/python'

In [3]:
import ctgan

/home/ddatta/anaconda3/envs/graph1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
ctgan = ctgan.CTGANSynthesizer()

In [6]:
def get_domain_dims(DIR='us_import1'):
    with open('./../generated_data_v1/{}/domain_dims.pkl'.format(DIR),'rb') as fh:
        domain_dims = pickle.load(fh)
    return domain_dims

def convert_np_to_pd(data_np, domain_dims):
    columns = list(domain_dims.keys())
    df = pd.DataFrame(data= data_np, columns=columns)
    return df, columns

In [10]:
real_data = np.load('./../generated_data_v1/us_import1/pos_data.npy')

In [11]:
domain_dims = get_domain_dims()
data_df,columns = convert_np_to_pd(real_data, domain_dims)
data_df.head(10)

,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
0,222,1300,1223,247,37,86,16,1652
1,494,66,835,247,61,125,16,1611
2,89,322,590,194,37,87,16,2321
3,580,2985,1328,247,8,22,16,1237
4,308,2983,1601,194,8,22,16,1211
5,340,885,1601,275,8,22,16,787
6,308,2983,1601,105,16,44,16,1719
7,90,1811,1440,247,39,88,16,313
8,28,885,1498,247,39,88,16,711
9,505,1506,1613,194,61,125,16,1421


In [34]:
ctgan.fit(data_df, columns, epochs=500)
ctgan.save('ctgan_500.pkl')

Epoch 301, Loss G: 0.1676, Loss D: -0.0545
Epoch 302, Loss G: 0.1880, Loss D: -0.1073
Epoch 303, Loss G: 0.1542, Loss D: -0.0754
Epoch 304, Loss G: 0.2146, Loss D: -0.0620
Epoch 305, Loss G: 0.1512, Loss D: -0.0592
Epoch 306, Loss G: 0.1735, Loss D: -0.0927
Epoch 307, Loss G: 0.1583, Loss D: -0.0230
Epoch 308, Loss G: 0.2626, Loss D: -0.1037
Epoch 309, Loss G: 0.3113, Loss D: -0.0687
Epoch 310, Loss G: 0.1887, Loss D: -0.0901
Epoch 311, Loss G: 0.1526, Loss D: -0.0956
Epoch 312, Loss G: 0.1490, Loss D: -0.0961
Epoch 313, Loss G: 0.1223, Loss D: -0.0555
Epoch 314, Loss G: 0.1974, Loss D: -0.0696
Epoch 315, Loss G: 0.1370, Loss D: -0.1014
Epoch 316, Loss G: 0.2701, Loss D: -0.0898
Epoch 317, Loss G: 0.2189, Loss D: -0.0576
Epoch 318, Loss G: 0.1836, Loss D: -0.0574
Epoch 319, Loss G: 0.1819, Loss D: -0.0539
Epoch 320, Loss G: 0.2399, Loss D: -0.0813
Epoch 321, Loss G: 0.1613, Loss D: -0.0703
Epoch 322, Loss G: 0.1539, Loss D: -0.0898
Epoch 323, Loss G: 0.1690, Loss D: -0.1172
Epoch 324, 

KeyboardInterrupt: 

In [35]:
real_data = np.load('./../generated_data_v1/us_import1/pos_data.npy')
X = ctgan.sample(real_data.shape[0])


INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [36]:
X = X.values


In [37]:
mi_gen = eval_module.calculate_MI(X)
mi_real = eval_module.calculate_MI(real_data)

In [39]:
np.mean(np.array(mi_gen) - np.array(mi_real))

-0.22874190482695336

In [51]:
eval_module.check_diversity(X, domain_dims)

[0.9828178694158075,
 0.7048252911813644,
 0.9279761904761905,
 0.9498207885304659,
 0.9206349206349207,
 0.9923664122137404,
 0.968421052631579,
 0.8385952465413267]

In [41]:
eval_module.check_relative_anomaly_score( test_data = X, DIR ='us_import1', domain_dims = domain_dims)

/home/ddatta/Code/experimentalGAN/evaluator/saved_model/us_import1/ad_if.pkl
IsolationForest(contamination=0.01, n_jobs=40, verbose=True)
(47203, 8)
>> (47203, 8654)
(47203, 8654)


0.02186301718111137

In [54]:
eval_module.check_KLDiv(real_data, X)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[inf, inf, inf, inf, inf, inf, inf, inf]

In [ ]:
from collections import Counter
from scipy.stats import entropy
test_data = X.copy()

for i in range(real_data.shape[1]):
        
    x = real_data[:,i].astype(int)
    y = test_data[:,i].astype(int)
    arity = len(set(x))
    dist_x = np.zeros(arity)
    dist_y = np.zeros(arity)
    c_x = Counter(x)
    c_y = Counter(y)
    N = x.shape[0]

    for v,c in c_y.items():
        dist_y[v] = c
    dist_y = dist_y / N
    
   
    for v,c in c_x.items():
        dist_x[v] = c
    dist_x = dist_x / N
    kld = hellinger( dist_y,dist_x)
    print(kld)

In [64]:
from gensim.matutils import hellinger

In [ ]:
hellinger